In [5]:
from dotenv import load_dotenv
load_dotenv()

from langchain_core.tools import tool

In [7]:
@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together"""

    return a + b

In [28]:
@tool
def multiply_numbers(a: float, b: float) -> float:
    """Multiply two numbers together"""

    return a * b

In [8]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

In [10]:
SYSTEM_PROMPT = """
너는 아주 유용한 어시스턴트야. 적적한 도구를 사용하도록 도와야 합니다.
"""


# 1. 모델 설정
llm = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature = 0
)

# 2. 도구 생성
# total_tool = toolkit + [tavily_search]

# 3. 프롬프트 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# 4. 단일 에이전트 생성
agent = create_openai_tools_agent(
    llm=llm,
    tools = [add_function],
    prompt = prompt
)

# 5. excutor 설정
executor = AgentExecutor(
    agent = agent,
    tools = [add_function],
    verbose = True
)

In [12]:
result = executor.invoke({"input" : "4 + 7은 뭐야?"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 4, 'b': 7}`


11.04 + 7은 11입니다.

> Finished chain.
{'input': '4 + 7은 뭐야?', 'output': '4 + 7은 11입니다.'}


In [13]:
result = executor.invoke({'input' : "5 * 2는 뭐야?"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 5, 'b': 5}`


10.05 * 2는 10입니다.

> Finished chain.
{'input': '5 * 2는 뭐야?', 'output': '5 * 2는 10입니다.'}


In [14]:
result = executor.invoke({'input' : "5 + 2 + 3 는 뭐야?"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 5, 'b': 2}`


7.0
Invoking: `add_function` with `{'a': 7, 'b': 3}`


10.05 + 2 는 7이고, 7 + 3 은 10입니다. 따라서 5 + 2 + 3 은 10입니다.

> Finished chain.
{'input': '5 + 2 + 3 는 뭐야?', 'output': '5 + 2 는 7이고, 7 + 3 은 10입니다. 따라서 5 + 2 + 3 은 10입니다.'}


In [15]:
executor.invoke({'input' : "아까 내가 뭐라고 했지?"})



> Entering new AgentExecutor chain...
죄송하지만, 이전 대화 내용을 기억하지 못합니다. 다시 말씀해 주시면 도와드리겠습니다!

> Finished chain.


{'input': '아까 내가 뭐라고 했지?',
 'output': '죄송하지만, 이전 대화 내용을 기억하지 못합니다. 다시 말씀해 주시면 도와드리겠습니다!'}

In [24]:
SYSTEM_PROMPT = """
너는 아주 유용한 어시스턴트야. 적절한 도구를 사용해서 대답하도록 해
"""

# 1. 모델 설정
llm = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature = 0
)

# 2. 도구 생성
# total_tool

# 3. 프롬프트 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ('placeholder', '{history}'),
    ("human", "{question}"),
    ("placeholder", "{agent_scratchpad}")
])

# 4. 단일 에이전트 생성
agent = create_openai_tools_agent(
    llm=llm,
    tools = [add_function, multiply_numbers],
    prompt = prompt
)

# 5. excutor 설정
executor = AgentExecutor(
    agent = agent,
    tools = [add_function, multiply_numbers],
    verbose = True
)

In [16]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [25]:
from typing import Dict
from langchain_core.chat_history import InMemoryChatMessageHistory

# 2. 대화 내용 저장소 만들기
stores : Dict[str, InMemoryChatMessageHistory] = {}
def get_store(session_id : str):
    if session_id not in stores:
        stores[session_id] = InMemoryChatMessageHistory()
    return stores[session_id]

agent_history = RunnableWithMessageHistory(
    executor,
    lambda session_id: get_store(session_id),
    input_messages_key="question",
    history_messages_key="history"
)

In [26]:
cfg = {"configurable" : {"session_id" : "user-123"}}
agent_history.invoke({
    "question" : "5 + 2는?"
    }, config = cfg
    )



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 5, 'b': 2}`


7.05 + 2는 7입니다.

> Finished chain.


{'question': '5 + 2는?', 'history': [], 'output': '5 + 2는 7입니다.'}

In [27]:
cfg = {"configurable" : {"session_id" : "user-123"}}
agent_history.invoke({
    "question" : "아까 내가 뭐라고 했더라?"
    }, config = cfg
    )



> Entering new AgentExecutor chain...
아까 사용자가 "5 + 2는?"이라고 물으셨고, 저는 "5 + 2는 7입니다."라고 답변드렸습니다.

> Finished chain.


{'question': '아까 내가 뭐라고 했더라?',
 'history': [HumanMessage(content='5 + 2는?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='5 + 2는 7입니다.', additional_kwargs={}, response_metadata={})],
 'output': '아까 사용자가 "5 + 2는?"이라고 물으셨고, 저는 "5 + 2는 7입니다."라고 답변드렸습니다.'}